##### Copyright 2020 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 正規化

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/addons/tutorials/layers_normalizations"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/addons/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/addons/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/addons/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>


## 概要

このノートブックでは、TensorFlow の[正規化レイヤー](https://github.com/tensorflow/addons/blob/master/tensorflow_addons/layers/normalizations.py)について簡単に説明します。現在、サポートされているレイヤーは次の通りです。

- **グループ正規化** (TensorFlow Addons)
- **インスタンス正規化** (TensorFlow Addons)
- **レイヤー正規化** (TensorFlow Core)

これらのレイヤーの基本的な考え方は、活性化レイヤーの出力を正規化してトレーニング中の収束性を向上させることです。[バッチ正規化](https://keras.io/layers/normalization/)とは対照的に、これらの正規化はバッチではなく、その代わりに単一サンプルのアクティブ化を正規化するので、再帰性ニューラルネットワークにも適しています。

通常、正規化は入力テンソル内のサブグループの平均と標準偏差を計算することによって実行されます。これにスケールとオフセット係数を適用することも可能です。

$y_{i} = \frac{\gamma ( x_{i} - \mu )}{\sigma }+ \beta$

$ y$ : 出力

$x$ : 入力

$\gamma$ : スケール係数

$\mu$: 平均

$\sigma$: 標準偏差

$\beta$: オフセット係数

次の画像は、これらの手法の違いを説明しています。各サブプロットは入力テンソルを示し、N をバッチ軸、C をチャンネル軸、(H、W) を空間軸（例えば画像の高さと幅）としています。青色のピクセルは、同じ平均と分散で正規化され、それらのピクセルの値を集約することによって計算されています。

![](https://github.com/shaohua0116/Group-Normalization-Tensorflow/raw/master/figure/gn.png)

出典: （https://arxiv.org/pdf/1803.08494.pdf）

重み、ガンマ、ベータはすべての正規化レイヤーでトレーニング可能なため、表現力が失われる可能性を補います。これらの要素を有効化するには、`center`または`scale`フラグを`True`に設定します。もちろん、`initializers`、`constraints`、`regularizer`を`beta`と`gamma`に使用して、トレーニングプロセス中にこれらの値を調整することができます。 

## セットアップ

### Tensorflow 2.0 と Tensorflow Addons をインストールする

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

### データセットを準備する

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## グループ正規化のチュートリアル

### 説明

グループ正規化は、入力チャンネルをより小さなサブグループに分割し、その平均と分散に基づいてこれらの値を正規化します。グループ正規化は単一の例を正規化するので、この手法はバッチサイズに依存しません。

グループ正規化は、実験では、画像分類タスクでバッチ正規化に近い結果を出しました。全体的なバッチサイズが小さく、バッチ正規化のパフォーマンスが悪くなるような場合には、バッチ正規化の代わりにグループ正規化を使用すると効果がある可能性があります。

###例: 標準の "channels last" 設定で Conv2D レイヤーの後の 10 個のチャンネルを 5 つのサブグループに分割します。

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # Groupnorm Layer
  tfa.layers.GroupNormalization(groups=5, axis=3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## インスタンス正規化のチュートリアル

### 前書き

インスタンス正規化は、グループサイズがチャンネルサイズ（または軸サイズ）と同じサイズである、グループ正規化の特殊なケースです。

実験の結果、バッチ正規化の代わりにインスタンス正規化を用いると、スタイル転送が良好に行われることが示されました。最近ではバッチ正規化の代替として、GAN でもインスタンス正規化が使用されるようになりました。

### 例

Conv2D レイヤーの後に InstanceNormalization を適用して、一様に初期化されたスケールとオフセット係数を使用します。

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # LayerNorm Layer
  tfa.layers.InstanceNormalization(axis=3, 
                                   center=True, 
                                   scale=True,
                                   beta_initializer="random_uniform",
                                   gamma_initializer="random_uniform"),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## レイヤー正規化のチュートリアル

### 前書き

レイヤー正規化は、グループサイズが 1 であるグループ正規化の特殊なケースです。単一サンプルのすべての活性化から平均と標準偏差を計算します。

実験の結果、レイヤーの正規化はバッチサイズに依存しないため、再帰型ニューラルネットワークに適していることが分かりました。

### 例

Conv2D レイヤーの後に Layernormalization を適用して、スケールとオフセット係数を使用します。 

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # LayerNorm Layer
  tf.keras.layers.LayerNormalization(axis=3 , center=True , scale=True),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## 文献

[レイヤー正規化](https://arxiv.org/pdf/1607.06450.pdf)

[インスタンス正規化](https://arxiv.org/pdf/1607.08022.pdf)

[グループ正規化](https://arxiv.org/pdf/1803.08494.pdf)